### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [2]:
from datetime import datetime

PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}


class ReadableEntity:
    
    instances = list()
    
    def __init__(self, title, author, genre, num_pages, page_format, rarity_index):
        self.title = title
        self.author = author
        self.genre = genre
        self.num_pages = num_pages
        self.page_format = page_format
        self.rarity_index = rarity_index
        self.__class__.instances.append(self)
    
    @staticmethod
    def get_rectangle_square(height, width):
        return height*width


class Journal(ReadableEntity):
    
    def __init__(self, issue_number, **kwargs):
        super().__init__(**kwargs)
        self.issue_number = issue_number
        
    @property
    def square_of_entity(self):
        height, width = PAGES_FORMAT[self.page_format.upper()]
        return ReadableEntity.get_rectangle_square(height, width)*self.num_pages

        
class Book(ReadableEntity):
    
    def __init__(self, publication_year, **kwargs):
        super().__init__(**kwargs)
        self.publication_year = publication_year

    @property
    def square_of_entity(self):
        height, width = PAGES_FORMAT[self.page_format.upper()]
        return ReadableEntity.get_rectangle_square(height, width)*self.num_pages

        

class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'a') as f:
            f.write("publication add datetime: "+str(datetime.now())+"\n")
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key])+"\n")
            f.write("\n")
     
    
class LibraryJournal(Journal, Exporter):
    
    def __init__(self, rare_publications_file_name, **kwargs):
        super().__init__(**kwargs)
        
        if self.rarity_index > 8:
            self.export_to_txt(rare_publications_file_name)


class LibraryBook(Book, Exporter):

    def __init__(self, rare_publications_file_name, **kwargs):
        super().__init__(**kwargs)
        
        if self.rarity_index > 8:
            self.export_to_txt(rare_publications_file_name)
            

In [3]:
! echo "" > rare_publications_file.txt

In [4]:
book1 = LibraryBook(title="Python для менеджеров", author="me", genre="fantastic", num_pages=300, 
                    page_format="A4", rarity_index=3, publication_year=2016,
                    rare_publications_file_name="rare_publications_file.txt")

book2 = LibraryBook(title="Advanced python за неделю",
                    author="John", genre="documentation", num_pages=int(1e5),
                    page_format="A4", rarity_index=9, publication_year=1999,
                    rare_publications_file_name="rare_publications_file.txt")

jour1 = LibraryJournal(title="Что делать если тебе нравятся стикеры?",
                       author="Коуч", genre="psychology", num_pages=12,
                       page_format="A4", rarity_index=3, issue_number=16,
                       rare_publications_file_name="rare_publications_file.txt")

jour2 = LibraryJournal(title="Учился на программиста а стал Аgile коучем", 
                       author="Отчисленный c ФКН", genre="failures", num_pages=10,
                       page_format="A4", rarity_index=10, issue_number=93,
                       rare_publications_file_name="rare_publications_file.txt")

In [5]:
for publication in ReadableEntity.instances:
    print(publication.title, publication.square_of_entity)

Python для менеджеров 18711000
Advanced python за неделю 6237000000
Что делать если тебе нравятся стикеры? 748440
Учился на программиста а стал Аgile коучем 623700


In [6]:
! cat rare_publications_file.txt


publication add datetime: 2019-11-03 11:42:37.083415
title: Advanced python за неделю
author: John
genre: documentation
num_pages: 100000
page_format: A4
rarity_index: 9
publication_year: 1999

publication add datetime: 2019-11-03 11:42:37.084188
title: Учился на программиста а стал Аgile коучем
author: Отчисленный c ФКН
genre: failures
num_pages: 10
page_format: A4
rarity_index: 10
issue_number: 93



### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [38]:
import sys
import gc


def get_real_object_size(obj, obj_ids=None):
    
    # не влючать ссылки на классы
    if isinstance(obj, type):
        return 0
    
    if obj_ids is None:
        obj_ids = set()
    
    # не считаем один и тот же объект 2 раза
    if id(obj) in obj_ids:
        return 0
    
    else:
        print("include: ", obj)
        obj_ids.add(id(obj))
        obj_ref = gc.get_referents(obj)

        # ссылается ли объект на другие объекты
        if len(obj_ref) == 0:
            return sys.getsizeof(obj)

        else:
            ref_size = sum(get_real_object_size(x, obj_ids) for x in obj_ref)
            return sys.getsizeof(obj) + ref_size

In [58]:
y = list((1, 2))
x = tuple((y, y, 1, "sdf"))

print("Total: ", get_real_object_size(x))

include:  ([1, 2], [1, 2], 1, 'sdf')
include:  sdf
include:  1
include:  [1, 2]
include:  2
Total:  292


In [59]:
class TestRefs(list):
    
    def __init__(self, array):
        self.array = array
        
x = TestRefs([1,1,3])

In [60]:
# не включая классы

get_real_object_size(x)

include:  []
include:  {'array': [1, 1, 3]}
include:  [1, 1, 3]
include:  3
include:  1


336

### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [189]:
class CoefList(list):
    """ Returns 0 if key greater that array size and can add (subtract) another list."""
    
    def __getitem__(self, key):
        
        if isinstance(key, slice):
            return CoefList(super().__getitem__(key))
        
        if (key >= self.__len__()) or (-key > self.__len__()):
            return 0
        else:
            return super().__getitem__(key)
        
    def add_list(self, another):
        return CoefList([self[i] + another[i]
                for i in range(-max(len(self), len(another)), 0)])
    
    def sub_list(self, another):
        return CoefList([self[i] - another[i]
                for i in range(-max(len(self), len(another)), 0)])
    
    def __add__(self, another):
        return CoefList(super().__add__(another))

In [326]:
class Polynomial():
    
    def __init__(self, degree_coefs):
        
        if not isinstance(degree_coefs, list):
            raise TypeError("degrees_coefs должен иметь тип list (а не {})".format(type(degree_coefs)))
        
        self.degree_coefs = CoefList(degree_coefs)
                
    def __add__(self, another):
        return self.degree_coefs.add_list(another.degree_coefs)
    
    def __sub__(self, another):
        return self.degree_coefs.sub_list(another.degree_coefs)
        
    
    @staticmethod
    def karatsuba(x, y):
        """
            Karatsuba Multiplication:

            x = x1*B + x0
            y = y1*B + y0

            z2 = x1*y1
            z0 = x0*y0
            z1 = (x1+x0)*(y1+y0) - z0 - z2

            result = z2*(B**2) + z1*B + z0
            
        Input:
            x, y: CoefList
             коэффициенты многочленов
        Return:
            CoefList, коэффициенты итогового многочлена
        
        """
        if not isinstance(x, CoefList):
            raise TypeError("x должен иметь тип CoefList (а не {})".format(type(x)))
        if not isinstance(y, CoefList):
            raise TypeError("y должен иметь тип CoefList (а не {})".format(type(y)))
        
        if len(x)==1:
            return CoefList([x[0]*y[i] for i in range(len(y))])
        if len(y)==1:
            return CoefList([y[0]*x[i] for i in range(len(x))])
        
        split_id = min(map(len,[x,y]))//2
        x0, x1 = x[-split_id:], x[:-split_id]
        y0, y1 = y[-split_id:], y[:-split_id]

        z2 = Polynomial.karatsuba(x1, y1)
        z0 = Polynomial.karatsuba(x0, y0)
        z1 = Polynomial.karatsuba(x1.add_list(x0), y1.add_list(y0))
        z1 = z1.sub_list(z2).sub_list(z0)
                
        z2 = z2 + [0]*(2*split_id)
        z1 = z1 + [0]*(split_id)
        
        
        return z2.add_list(z1).add_list(z0)
        
    def __mul__(self, another):
        new_coefs = Polynomial.karatsuba(self.degree_coefs, another.degree_coefs)
        return Polynomial(new_coefs)
    
    def __

### сумма, вычитание

In [327]:
x = Polynomial([1,3,4])
y = Polynomial([4,3,4,0])
x+y, x-y

([4, 4, 7, 4], [-4, -2, -1, 4])

### Тест умножения 

In [328]:
import numpy as np

def correct_mul(x, y):
    x = np.array(x)
    y = np.array(y)
    
    matrix = x[:,None]*y[None,:]
    result = list()
    
    left_border = matrix.shape[0]-1
    right_border = matrix.shape[1]
    for i in range(-left_border, right_border):
        result.append(np.diag(matrix[::-1],i).sum())

    return result

correct_mul([4,2,1,0], [0,1,2,3])

[0, 4, 10, 17, 8, 3, 0]

In [329]:
for i in range(100):
    len_x, len_y = np.random.randint(1,10,2)
    x = np.random.randint(0,10, len_x).tolist()
    y = np.random.randint(0,10, len_y).tolist()
    
    res1 = Polynomial.karatsuba(CoefList(x), CoefList(y))
    res2 = correct_mul(x, y)
    
    if res1 == res2:
        print("-"*10)
        print("OK")
    else:
        print("-"*10)
        print("x: ", x)
        print("y: ", y)
        print(res1)
        print(res2)

----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
----------
OK
------

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [49]:
def range_like_iterator(n):
    i = 0
    while i < n:
        yield i
        i +=1

In [54]:
list(range_like_iterator(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [45]:
from datetime import datetime


class Timer():
    
    def __enter__(self):
        self.start_time = datetime.now()
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(datetime.now() - self.start_time)

In [48]:
with Timer() as t:
    x = list(range(10))

0:00:00.000045
